## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [35]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_titles():
    col = cities.columns
    col = [re.split('\[',title)[0] for title in col]
    cities.columns = col
    for col in cities.columns[2:]:
        cities[col] = [re.split('\[[\w\s\d]*\]',city)[0] for city in cities[col]]

clean_titles()

    #cities[col] = [', '.join(re.findall('[A-Z][^A-Z]*',team)) for team in cities[col]]
    
def clean_nhl():
    df = nhl_df[nhl_df['year']==2018].iloc[:,[0,2,3]]
    df['team'] = [re.split('\*',team)[0] for team in df['team']]
    df.drop([0,9,18,26], inplace=True)
    df['W'] = df['W'].astype(int)
    df['L'] = df['L'].astype(int)
    df['ratio'] = df['W']/(df['W']+df['L'])
    return df

def nhl_correlation(): 
    
    population_by_region = list(df1['Population (2016 est.)'].to_numpy()) # pass in metropolitan area population from cities
    win_loss_by_region = list(df1['ratio'].to_numpy()) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    return stats.pearsonr(population_by_region, win_loss_by_region)

nhl = cities.iloc[:,[0,1,-1]]
nhl = nhl.replace('—',np.nan,regex=True).dropna()
nhl = nhl.mask(nhl['NHL']=='').dropna()
nhl['Population (2016 est.)'] = nhl['Population (2016 est.)'].astype(int)

nhl_wl = clean_nhl()

names = []
areas = []

for pair in nhl.iloc[:,[0,2]].itertuples(index=False, name=None):
    names.append(pair[0]+', '+pair[1])

for team in nhl_wl['team']:
    for name in names:
        if team.split()[-1] in name:
            areas.append(name.split(',')[0])

areas.insert(areas.index('Washington'), 'Washington, D.C.')
areas.remove('Washington')
nhl_wl['area'] = areas

nhl = nhl.sort_values('Metropolitan area').set_index('Metropolitan area')
nhl_wl = nhl_wl.sort_values('area').set_index('area')

df1 = pd.merge(nhl_wl, nhl, left_index=True, right_index=True)
df1 = df1.groupby(level=0).agg({'ratio':np.mean,'Population (2016 est.)':np.mean})
df1.head()

,ratio,Population (2016 est.)
Boston,0.714286,4794447
Buffalo,0.357143,1132804
Calgary,0.513889,1392609
Chicago,0.458333,9512999
Columbus,0.600000,2041520


In [29]:
clean_nhl().head()

,team,W,L,ratio
1,Tampa Bay Lightning,54,23,0.701299
2,Boston Bruins,50,20,0.714286
3,Toronto Maple Leafs,49,26,0.653333
4,Florida Panthers,44,30,0.594595
5,Detroit Red Wings,30,39,0.434783


In [8]:
nhl_correlation()

(0.012486162921209895, 0.9497182859911791)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [30]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

clean_titles()

def clean_nba():
    df = nba_df[nba_df['year']==2018].iloc[:,[0,1,2]]
    df['team'] = [re.split('[\*\(]',team)[0] for team in df['team']]
    df['W'] = df['W'].astype(int)
    df['L'] = df['L'].astype(int)
    df['ratio'] = df['W']/(df['W']+df['L'])
    return df

def nba_correlation():

    population_by_region = list(df2['Population (2016 est.)'].to_numpy()) # pass in metropolitan area population from cities
    win_loss_by_region = list(df2['ratio'].to_numpy()) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    return stats.pearsonr(population_by_region, win_loss_by_region)

nba_wl = clean_nba()

nba = cities.iloc[:,[0,1,4]]
nba = nba.replace('—',np.nan,regex=True).dropna()
nba = nba.mask(nba['NBA']=='').dropna()
nba['Population (2016 est.)'] = nba['Population (2016 est.)'].astype(int)

names = []
areas = []
for pair in nba.iloc[:,[0,2]].itertuples(index=False, name=None):
    names.append(pair[0]+', '+pair[1])

for team in nba_wl['team']:
    for name in names:
        if team.split()[-1] in name:
            areas.append(name.split(',')[0])
            
areas.insert(areas.index('Washington'), 'Washington, D.C.')
areas.remove('Washington')

nba_wl['area'] = areas

nba = nba.sort_values('Metropolitan area').set_index('Metropolitan area')
nba_wl = nba_wl.sort_values('area').set_index('area')

df2 = pd.merge(nba_wl, nba, left_index=True, right_index=True)
df2 = df2.groupby(level=0).agg({'ratio':np.mean,'Population (2016 est.)':np.mean})
df2.head()

,ratio,Population (2016 est.)
Atlanta,0.292683,5789700
Boston,0.670732,4794447
Charlotte,0.439024,2474314
Chicago,0.329268,9512999
Cleveland,0.609756,2055612


In [31]:
clean_nba().head()

,team,W,L,ratio
0,Toronto Raptors,59,23,0.719512
1,Boston Celtics,55,27,0.670732
2,Philadelphia 76ers,52,30,0.634146
3,Cleveland Cavaliers,50,32,0.609756
4,Indiana Pacers,48,34,0.585366


In [13]:
nba_correlation()

(-0.17657160252844611, 0.3687474160446298)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [32]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

clean_titles()

def clean_mlb():
    df = mlb_df[mlb_df['year']==2018].iloc[:,[0,1,2]]
    df['team'] = [re.split('[\*\(]',team)[0] for team in df['team']]
    df['W'] = df['W'].astype(int)
    df['L'] = df['L'].astype(int)
    df['ratio'] = df['W']/(df['W']+df['L'])
    return df

def mlb_correlation(): 

    population_by_region = list(df3['Population (2016 est.)'].to_numpy()) # pass in metropolitan area population from cities
    win_loss_by_region = list(df3['ratio'].to_numpy()) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    return stats.pearsonr(population_by_region, win_loss_by_region)

mlb_wl = clean_mlb()

mlb = cities.iloc[:,[0,1,3]]
mlb = mlb.replace('—',np.nan,regex=True).dropna()
mlb = mlb.mask(mlb['MLB']=='').dropna()
mlb['Population (2016 est.)'] = mlb['Population (2016 est.)'].astype(int)

names = []
areas = []
for pair in mlb.iloc[:,[0,2]].itertuples(index=False, name=None):
    names.append(pair[0]+', '+pair[1])

for team in mlb_wl['team']:
    for name in names:
        if team.split()[-1] in name:
            areas.append(name.split(',')[0])
            
areas.insert(areas.index('Washington'), 'Washington, D.C.')
areas.remove('Washington')
areas.remove('Chicago')
areas.insert(8, 'Chicago')
del areas[9:11]
mlb_wl['area'] = areas

mlb = mlb.sort_values('Metropolitan area').set_index('Metropolitan area')
mlb_wl = mlb_wl.sort_values('area').set_index('area')

df3 = pd.merge(mlb_wl, mlb, left_index=True, right_index=True)
df3 = df3.groupby(level=0).agg({'ratio':np.mean,'Population (2016 est.)':np.mean})
df3.head()

,ratio,Population (2016 est.)
Atlanta,0.555556,5789700
Baltimore,0.290123,2798886
Boston,0.666667,4794447
Chicago,0.482769,9512999
Cincinnati,0.413580,2165139


In [34]:
clean_mlb().head()

,team,W,L,ratio
0,Boston Red Sox,108,54,0.666667
1,New York Yankees,100,62,0.617284
2,Tampa Bay Rays,90,72,0.555556
3,Toronto Blue Jays,73,89,0.450617
4,Baltimore Orioles,47,115,0.290123


In [17]:
mlb_correlation()

(0.15027698302669307, 0.46370703378875583)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [36]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

clean_titles()

def clean_nfl():
    df = nfl_df[nfl_df['year']==2018].iloc[:,[13,11,1]]
    df = df.drop(index=list(df.iloc[0::5].index))
    df['team'] = [re.split('[\*\+]',team)[0] for team in df['team']]
    df['W'] = df['W'].astype(int)
    df['L'] = df['L'].astype(int)
    df['ratio'] = df['W']/(df['W']+df['L'])
    return df

def nfl_correlation():

    population_by_region = list(df4['Population (2016 est.)'].to_numpy()) # pass in metropolitan area population from cities
    win_loss_by_region = list(df4['ratio'].to_numpy()) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    return stats.pearsonr(population_by_region, win_loss_by_region)

nfl_wl = clean_nfl()

nfl = cities.iloc[:,[0,1,2]]
nfl = nfl.replace('—',np.nan,regex=True).dropna()
nfl = nfl.mask(nfl['NFL']=='').dropna()
nfl['Population (2016 est.)'] = nfl['Population (2016 est.)'].astype(int)

names = []
areas = []
for pair in nfl.iloc[:,[0,2]].itertuples(index=False, name=None):
    names.append(pair[0]+', '+pair[1])

for team in nfl_wl['team']:
    for name in names:
        if team.split()[-1] in name:
            areas.append(name.split(',')[0])
            
areas.insert(areas.index('Washington'), 'Washington, D.C.')
areas.remove('Washington')

nfl_wl['area'] = areas

nfl = nfl.sort_values('Metropolitan area').set_index('Metropolitan area')
nfl_wl = nfl_wl.sort_values('area').set_index('area')

df4 = pd.merge(nfl_wl, nfl, left_index=True, right_index=True)
df4 = df4.groupby(level=0).agg({'ratio':np.mean,'Population (2016 est.)':np.mean})
df4.head()

,ratio,Population (2016 est.)
Atlanta,0.4375,5789700
Baltimore,0.6250,2798886
Boston,0.6875,4794447
Buffalo,0.3750,1132804
Charlotte,0.4375,2474314


In [38]:
clean_nfl().head()

,team,W,L,ratio
1,New England Patriots,11,5,0.6875
2,Miami Dolphins,7,9,0.4375
3,Buffalo Bills,6,10,0.3750
4,New York Jets,4,12,0.2500
6,Baltimore Ravens,10,6,0.6250


In [19]:
nfl_correlation()

(0.004922112149349428, 0.9797833458363694)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_rel
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    
    MLB = df3.drop(columns=['Population (2016 est.)']).rename(columns={'ratio':'MLB'})
    NBA = df2.drop(columns=['Population (2016 est.)']).rename(columns={'ratio':'NBA'})
    NHL = df1.drop(columns=['Population (2016 est.)']).rename(columns={'ratio':'NHL'})
    NFL = df4.drop(columns=['Population (2016 est.)']).rename(columns={'ratio':'NFL'})
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    dfs = [NFL, NBA, NHL, MLB]
    for index in range(len(sports)):
        for column in range(len(sports)):
            if index != column:
                df = pd.merge(dfs[index], dfs[column], left_index=True, right_index=True)
                p_values.iloc[index,column] = ttest_rel(df.iloc[:,0].to_numpy(), df.iloc[:,1].to_numpy())[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values



In [24]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
